<a href="https://colab.research.google.com/github/iremaricii/SmartbookAI/blob/main/smartbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn sentence-transformers gradio


In [ ]:
# Veri setini indir
!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv

import pandas as pd

# Kitap verisini oku
df = pd.read_csv("books.csv")

# Gerekli sütunları seç
df = df[['title', 'authors', 'average_rating', 'language_code']]
df = df[df['language_code'] == 'eng'].dropna().reset_index(drop=True)

# Embedding için metin oluştur
df["text"] = df["title"] + " by " + df["authors"]

df[["title", "authors", "average_rating", "text"]].head()


In [9]:
from sentence_transformers import SentenceTransformer, util
import torch

# Modeli yükle
model = SentenceTransformer('all-MiniLM-L6-v2')

# Tüm kitap açıklamalarını encode et
book_embeddings = model.encode(df["text"].tolist(), convert_to_tensor=True)


In [ ]:
# Kullanıcıdan örnek metin
kullanici_girdisi = "I like science fiction and psychological books. I'm feeling a bit stressed but focused today."

# Kullanıcı metnini encode et
kullanici_embedding = model.encode(kullanici_girdisi, convert_to_tensor=True)

# Benzerlik hesapla
cosine_scores = util.cos_sim(kullanici_embedding, book_embeddings)[0]

# En benzer 3 kitabı bul
top_k = torch.topk(cosine_scores, k=3)

# Sonuçları yazdır
print("📚 Önerilen Kitaplar:\n")
for score, idx in zip(top_k.values, top_k.indices):
    i = idx.item()
    print(f"{df.iloc[i]['title']} by {df.iloc[i]['authors']} — Skor: {score:.4f}")


In [ ]:
# Kullanıcının günlük okuma süresi (dakika)
gunluk_okuma_suresi = 20

# Ortalama kitap uzunluğu (sayfa)
ortalama_sayfa_sayisi = 300

# Okuma hızı: 1 sayfa/dk → toplam dakika = 300
toplam_okuma_suresi_dk = ortalama_sayfa_sayisi
gunluk_okuma_suresi_dk = gunluk_okuma_suresi

# Tahmini gün sayısı
tahmini_gun = toplam_okuma_suresi_dk / gunluk_okuma_suresi_dk

print(f"\n📖 Ortalama bir kitabı günde {gunluk_okuma_suresi_dk} dakika okuyarak yaklaşık {round(tahmini_gun)} günde bitirebilirsin.")


In [16]:
def kitap_oner(ilgi_alanlari, ruh_hali, gunluk_dk):
    if not ilgi_alanlari:
        return "📌 Lütfen en az bir ilgi alanı seç!", None, None, None

    kullanici_girdisi = f"I like {', '.join(ilgi_alanlari)} books. I'm feeling {ruh_hali.lower()} today."
    kullanici_embedding = model.encode(kullanici_girdisi, convert_to_tensor=True)
    cosine_scores = util.cos_sim(kullanici_embedding, book_embeddings)[0]
    top_k = torch.topk(cosine_scores, k=3)

    kitap_listesi = ""
    kapaklar = []
    kitap_adlari = []

    for score, idx in zip(top_k.values, top_k.indices):
        i = idx.item()
        kitap_adlari.append(df.iloc[i]['title'])
        kitap_listesi += f"📘 <b>{df.iloc[i]['title']}</b> by <i>{df.iloc[i]['authors']}</i><br>Skor: <b>{score:.4f}</b><br><br>"
        kapaklar.append(df.iloc[i]['image_url'])

    tahmini_gun = round(300 / max(1, gunluk_dk))

    sonuc = f"""\
<div style="background-color:#F9F9F9; padding: 10px; border-radius: 12px;">
<h3>📚 Önerilen Kitaplar</h3>
{kitap_listesi}
<hr>
<h4>⏱ Okuma Süresi Tahmini</h4>
Günde <b>{gunluk_dk}</b> dakika okuyarak bu kitapları yaklaşık <b>{tahmini_gun} günde</b> bitirebilirsin.
</div>
"""
    return sonuc, kapaklar[0], kapaklar[1], kapaklar[2]


In [ ]:
import gradio as gr
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

# Veri yükleme
df = pd.read_csv("books.csv")
df = df[['title', 'authors', 'average_rating', 'language_code', 'image_url']]
df = df[df['language_code'] == 'eng'].dropna().reset_index(drop=True)
df["text"] = df["title"] + " by " + df["authors"]

# Model yükleme
model = SentenceTransformer('all-MiniLM-L6-v2')
book_embeddings = model.encode(df["text"].tolist(), convert_to_tensor=True)

# Öneri fonksiyonu (kapak + kitap bir arada)
def kitap_oner(ilgi_alanlari, ruh_hali, gunluk_dk):
    if not ilgi_alanlari:
        return "<p style='color:red;'>📌 Lütfen en az bir ilgi alanı seç!</p>"

    kullanici_girdisi = f"I like {', '.join(ilgi_alanlari)} books. I'm feeling {ruh_hali.lower()} today."
    kullanici_embedding = model.encode(kullanici_girdisi, convert_to_tensor=True)
    cosine_scores = util.cos_sim(kullanici_embedding, book_embeddings)[0]
    top_k = torch.topk(cosine_scores, k=3)

    # Kartları HTML ile oluştur
    kart_html = ""
    for score, idx in zip(top_k.values, top_k.indices):
        i = idx.item()
        title = df.iloc[i]['title']
        author = df.iloc[i]['authors']
        image_url = df.iloc[i]['image_url']
        skor = f"{score:.4f}"

        kart_html += f"""
        <div style="flex: 1; margin: 10px; padding: 10px; border-radius: 12px;
            background-color: #f9f9f9; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); text-align:center;">
            <img src="{image_url}" style="width:110px; height:160px; border-radius:8px; box-shadow: 1px 1px 5px #ccc;"><br><br>
            <b>{title}</b><br>
            <i>{author}</i><br>
            📊 <b>Skor:</b> {skor}
        </div>
        """

    tahmini_gun = round(300 / max(1, gunluk_dk))

    sonuc_html = f"""
    <div style="margin-top: 20px;">
        <h3 style="text-align:center; color:#5E60CE;">📚 Önerilen Kitaplar</h3>
        <div style="display:flex; justify-content: space-around;">{kart_html}</div>
        <hr>
        <p style="text-align:center;">⏱ Günde <b>{gunluk_dk}</b> dakika okuyarak bu kitapları yaklaşık
        <b>{tahmini_gun} günde</b> bitirebilirsin.</p>
    </div>
    """

    return sonuc_html

# İlgi alanı ve ruh hali seçenekleri
kategori_listesi = [
    "📖 Science Fiction", "💘 Romance", "🏰 Historical", "🧩 Thriller",
    "🐉 Fantasy", "🧠 Psychology", "🌱 Self-Help", "👤 Biography",
    "🕵️ Mystery", "🌍 Adventure"
]

ruh_halleri = ["😊 Happy", "😞 Sad", "🎯 Focused", "😬 Anxious", "🚀 Motivated", "😴 Tired"]

# Gradio Arayüzü
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """<div style='text-align: center'>
        <h1 style='color: #5E60CE;'>📘 SmartBook AI</h1>
        <p style='font-size: 16px;'>💡 İlgi alanlarına ve ruh haline göre kitap önerisi yapar,<br>
        ve okuma süresini tahmin eder.</p>
        </div>"""
    )

    with gr.Row():
        with gr.Column():
            ilgi = gr.CheckboxGroup(choices=kategori_listesi, label="✨ İlgi Alanlarını Seç", info="Birden fazla seçebilirsin.")
            ruh = gr.Dropdown(choices=ruh_halleri, label="🌈 Bugünkü Ruh Halin")
            sure = gr.Slider(5, 120, value=30, step=5, label="⏱ Günlük Okuma Süren (dk)")
            buton = gr.Button("🔍 Bana Kitap Öner!")

        with gr.Column():
            sonuc = gr.HTML(label="📘 Önerilen Kitaplar")

    buton.click(fn=kitap_oner, inputs=[ilgi, ruh, sure], outputs=[sonuc])

demo.launch()
